## 3. Path Planning

The goal of this section is to find the shortest path between two points in a grid-like structure while minimizing the total cost associated with the path. <br /><br />
We chose to use the A* Algorithm because it it garanteed to find the shortest path and it is more efficient than other algorithms in terms of both time and memory usage.<br/><br />
We use this algorithm to find the shortest path from the robot initial position to the robot targeted position. At this point of the project we are taking into account only the global obstacles visible from the camera, and not the local ones.


*<u>Image 3.1:</u>* _Example of the shortest path from the Thymio robot (top left) to the target (bottom right) using A* Algorithm._

![a-star-path](images/a-star-path-example.jpg)


The Node class initializes each node with its coordinates (coord) and the goal node's coordinates (goal).
- The g value is the cost from the start node to the current node, initialized to infinity.
- The _h value is the heuristic estimate  of the cost to reach the goal from the current node.
- The _f value is the sum of g and _h, used to determine the node's priority in the pathfinding process.
- The set_cost and set_parent methods are used to update a node's cost and its parent in the path.
- The _get_dist method calculates the heuristic distance (Manhattan distance) between two nodes, considering both straight and diagonal movements.

The A_star class is initialized with a grid and a set of camera coordinates.
- We chose 8 possible movements directions from a node.
- The _path_constructor method reconstructs the path from the start to the goal node, traversing backward from the goal.
- The _is_traversible method checks if a coordinate is within the grid and thus accessible.
- The find_path method implements the core A* algorithm. It uses a priority queue (heapq) to efficiently select the next node to explore. The method returns the constructed path from start to goal if available, or an empty list if no path is found.